In [8]:
import numpy as np
import pandas as pd
import os
import glob

def merge_strategy_model_npy(base_dir, strategy, model, output_csv="combined.csv"):
    """
    Merge all .npy files under a given strategy/model path into one CSV.
    
    Args:
        base_dir (str): Base directory containing strategy folders (e.g. 'saved_residuals').
        strategy (str): Strategy name (e.g. 'rolling', 'online', 'online_incremental').
        model (str): Model name under strategy folder (e.g. 'hinge', 'hinge_weibull').
        output_csv (str): Name of the output CSV file.
    """
    folder_path = os.path.join(base_dir, strategy, model)
    npy_files = sorted(glob.glob(os.path.join(folder_path, '*.npy')))

    all_data = []

    for file in npy_files:
        data = np.load(file).flatten()
        interval_name = os.path.splitext(os.path.basename(file))[0]
        df = pd.DataFrame({
        "interval": [interval_name] * len(data),
        "value": data
    })
        all_data.append(df)

    combined_df = pd.concat(all_data, ignore_index=True)
    combined_df.to_csv(output_csv, index=False)
    print(f"✅ Saved combined CSV to {output_csv}")


In [23]:
def merge_strategy_model_csv(base_dir, strategy, model, output_csv="combined.csv"):
    """
    Merge all CSV files under a given strategy/model path into one CSV.
    
    Args:
        base_dir (str): Base directory containing strategy folders (e.g. 'saved_residuals').
        strategy (str): Strategy name (e.g. 'rolling', 'online_monthly', 'online_expanding_monthly').
        model (str): Model name under strategy folder (e.g. 'hinge', 'hinge_weibull').
        output_csv (str): Name of the output CSV file.
    """
    folder_path = os.path.join(base_dir, strategy, model)
    csv_files = sorted(glob.glob(os.path.join(folder_path, 'residuals_*.csv')))
    
    if not csv_files:
        print(f"No CSV files found in {folder_path}")
        return

    all_data = []
    for file in csv_files:
        df = pd.read_csv(file)
        df['date'] = pd.to_datetime(df['date'])
        step_name = os.path.splitext(os.path.basename(file))[0]
        df['step'] = step_name
        all_data.append(df)

    combined_df = pd.concat(all_data, ignore_index=True)
    combined_df = combined_df.sort_values('date').reset_index(drop=True)
    
    # Keep only the required columns in the specified order
    final_df = combined_df[['date', 'residual', 'prediction']].copy()
    
    final_df.to_csv(output_csv, index=False)
    print(f"✅ Saved combined CSV to {output_csv} with {len(final_df)} rows")

model_list = ["normal", "hinge", "lognormal", "inverse_gaussian", "weibull", "hinge_weibull","hinge_inverse_gaussian"]
strategy = "online_monthly"

for model in model_list:
    output_filename = f"{model}_{strategy}.csv"
    merge_strategy_model_csv(
        base_dir="saved_residuals",
        strategy=strategy,
        model=model,
        output_csv=output_filename
    )

✅ Saved combined CSV to normal_online_monthly.csv with 1958 rows
✅ Saved combined CSV to hinge_online_monthly.csv with 1958 rows
✅ Saved combined CSV to lognormal_online_monthly.csv with 1958 rows
✅ Saved combined CSV to inverse_gaussian_online_monthly.csv with 1958 rows
✅ Saved combined CSV to weibull_online_monthly.csv with 1958 rows
✅ Saved combined CSV to hinge_weibull_online_monthly.csv with 1958 rows
✅ Saved combined CSV to hinge_inverse_gaussian_online_monthly.csv with 1958 rows


In [10]:
model_list = ["normal", "hinge", "lognormal", "inverse_gaussian", "weibull", "hinge_weibull", "hinge_inverse_gaussian"]

strategy = "online_expanding_monthly"

for model in model_list:
    output_filename = f"{model}_{strategy}.csv"
    merge_strategy_model_npy(
        base_dir="saved_residuals",
        strategy=strategy,
        model=model,
        output_csv=output_filename
    )

✅ Saved combined CSV to normal_online_expanding_monthly.csv
✅ Saved combined CSV to hinge_online_expanding_monthly.csv
✅ Saved combined CSV to lognormal_online_expanding_monthly.csv
✅ Saved combined CSV to inverse_gaussian_online_expanding_monthly.csv
✅ Saved combined CSV to weibull_online_expanding_monthly.csv
✅ Saved combined CSV to hinge_weibull_online_expanding_monthly.csv
✅ Saved combined CSV to hinge_inverse_gaussian_online_expanding_monthly.csv
